In [1]:
import numpy as np
#!pip install gurobipy
import gurobipy as gp
GRB = gp.GRB
import time

In [2]:
def flowshopwithtimelags(d, e, tlagsmin, tlagsmax):
    n = len(d[0])  # nombre de jobs
    m = len(d)     # nombre de machines

    # Modèle
    model = gp.Model("FlowshopWithTimeLags")


    # Variables de permutation : x[i,j] = 1 si job i est en position j
    x = model.addVars(n, n, vtype=GRB.BINARY)

    # Dates de fin C[i,k] pour chaque job i sur chaque machine k
    C = model.addVars(n, m, vtype=GRB.CONTINUOUS)

    # Variables de retard : 1 si le job i termine après son échéance
    retard = model.addVars(n, vtype=GRB.BINARY)

    # Contraintes de permutation
    for i in range(n):
        model.addConstr(gp.quicksum(x[i, j] for j in range(n)) == 1)  # Chaque job à une position
    for j in range(n):
        model.addConstr(gp.quicksum(x[i, j] for i in range(n)) == 1)  # Une seule job par position

    # Définition des C[i,0] : première machine
    for i in range(n):
        for j in range(n):
            for jp in range(n):
                if jp < j:
                    model.addConstr(C[i, 0] >= C[i, 0] + 0 * x[i, j] * x[i, jp])  # Pour respecter le style, mais inutile ici
        model.addConstr(C[i, 0] >= gp.quicksum(x[i, j] * (
            gp.quicksum(x[ip, jp] * d[0][ip] for ip in range(n) for jp in range(j) if jp < j)
        ) for j in range(n)))

    # Calcul de C[i,k] avec time lags, pour k > 0
    for i in range(n):
        for k in range(1, m):
            model.addConstr(C[i, k] >= C[i, k - 1] + d[k - 1][i] + tlagsmin[k][i])
            model.addConstr(C[i, k] <= C[i, k - 1] + d[k - 1][i] + tlagsmax[k][i])

    # Calcul du retard
    for i in range(n):
        model.addConstr(retard[i] >= (C[i, m - 1] - e[i]) / 10000)  # Relaxation linéaire
        model.addConstr(retard[i] <= 1)

    # Objectif : minimiser le nombre de jobs en retard
    model.setObjective(gp.quicksum(retard[i] for i in range(n)), GRB.MINIMIZE)

    # Résolution
    model.optimize()

    # Reconstitution de l'ordonnancement
    if model.status == GRB.OPTIMAL:
        ordre = [-1] * n
        for i in range(n):
            for j in range(n):
                if x[i, j].X > 0.5:
                    ordre[j] = i + 1  # Pour avoir une permutation de {1, ..., n}
        Ntmin = int(sum(retard[i].X for i in range(n)))
        return ordre, Ntmin
    else:
        raise Exception("Aucune solution optimale trouvée.")


In [3]:
def flowshopwithtimelagslexico(d,e,tlagsmin,tlagsmax):
  ...

  return ordonnancement, Ntmin, Cmaxmin

In [4]:
#DONNEES
# n = 20
# m = 5
# d = [[96,68,91,46,88,29,23,74,91,45,69,36,34,91,77,25,40,70,30,52],
#      [81,39,92,63,83,60,39,97,27,94,22,59,46,93,99,66,62,71,76,73],
#      [97,31,93,58,96,37,94,47,91,47,79,66,20,33,38,54,29,56,70,85],
#      [61,77,82,69,27,54,66,89,72,46,90,41,75,79,79,92,55,47,64,62],
#      [91,82,96,75,68,51,75,22,20,91,80,93,53,50,46,58,90,60,51,20]]
# e = [1377,1559,1384,1175,1489,1536,1275,1526,1433,1426,1159,1275,1280,1141,1081,1429,1366,1271,1282,1535]

# tlagsmin = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
#             [87,87,42,14,8,44,33,8,71,14,11,52,62,53,17,35,74,46,81,81],
#             [60,40,75,40,2,70,23,12,42,20,16,34,70,91,28,62,2,46,73,4],
#             [53,64,18,83,27,84,0,56,76,33,62,28,19,2,23,1,30,22,45,14],
#             [30,48,67,13,2,80,23,12,2,13,66,61,3,68,2,2,68,12,69,53]]
# tlagsmax = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
#            [116,98,96,38,68,50,115,67,109,36,34,54,76,87,96,87,87,98,119,109],
#            [103,110,76,86,93,91,29,55,58,96,66,55,92,117,33,113,11,60,88,23],
#            [84,118,63,97,39,91,106,82,105,77,68,76,67,93,97,9,56,60,77,79],
#            [84,86,82,80,81,102,48,73,51,81,75,86,39,99,65,93,86,84,98,84]]


# DONNÉES RÉDUITES (pour contourner la limite de taille de Gurobi)
n = 6
m = 5

d = [
    [96, 68, 91, 46, 88, 29],  # Machine 1
    [81, 39, 92, 63, 83, 60],  # Machine 2
    [97, 31, 93, 58, 96, 37],  # Machine 3
    [61, 77, 82, 69, 27, 54],  # Machine 4
    [91, 82, 96, 75, 68, 51],  # Machine 5
]

e = [1377, 1559, 1384, 1175, 1489, 1536]

tlagsmin = [
    [0, 0, 0, 0, 0, 0],        # Pas de time lag min pour la 1re machine
    [87, 87, 42, 14, 8, 44],   # Entre M1 et M2
    [60, 40, 75, 40, 2, 70],   # Entre M2 et M3
    [53, 64, 18, 83, 27, 84],  # Entre M3 et M4
    [30, 48, 67, 13, 2, 80],   # Entre M4 et M5
]

tlagsmax = [
    [0, 0, 0, 0, 0, 0],
    [116, 98, 96, 38, 68, 50],
    [103, 110, 76, 86, 93, 91],
    [84, 118, 63, 97, 39, 91],
    [84, 86, 82, 80, 81, 102],
]


In [5]:
start_time = time.time()
ordonnancement, Ntmin = flowshopwithtimelags(d,e,tlagsmin,tlagsmax)
end_time = time.time()-start_time

Set parameter Username
Set parameter LicenseID to value 2661355
Academic license - for non-commercial use only - expires 2026-05-06
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-1035G4 CPU @ 1.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 72 rows, 72 columns and 186 nonzeros
Model fingerprint: 0xb0e6cfe7
Model has 96 quadratic constraints
Variable types: 30 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  QMatrix range    [3e+01, 1e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 2e+02]
Presolve removed 24 rows and 6 columns
Presolve time: 0.01s
Presolved: 594 rows, 606 columns, 2322 nonzeros
Variable types: 24 continuous, 582 integer (582 binary)
Found heuristic solution: objective 0.0000000

In [6]:
start_time = time.time()
ordonnancement, Ntmin, Cmaxmin = flowshopwithtimelagslexico(d,e,tlagsmin,tlagsmax)
end_time = time.time()-start_time

NameError: name 'Cmaxmin' is not defined